# Report Generator

> Generate compliance reports for docments validation

In [ ]:
#| default_exp report

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Dict, Any, Optional
from pathlib import Path
import json
from cjm_nbdev_docments.core import DocmentsCheckResult, check_definition
from cjm_nbdev_docments.scanner import scan_project, scan_notebook
from collections import defaultdict

In [ ]:
#| export
def check_project(
    nbs_path: Optional[Path] = None  # Path to notebooks directory
) -> List[DocmentsCheckResult]:  # List of check results for all definitions
    "Check all exported definitions in a project for docments compliance"
    definitions = scan_project(nbs_path)
    results = []
    
    for defn in definitions:
        result = check_definition(defn)
        results.append(result)
    
    return results

In [ ]:
#| export
def _generate_summary_stats(
    results: List[DocmentsCheckResult]  # Check results to summarize
) -> List[str]:  # Lines of summary statistics
    "Generate summary statistics section of the report"
    compliant = [r for r in results if r.is_compliant]
    non_compliant = [r for r in results if not r.is_compliant]
    with_todos = [r for r in results if r.has_todos]
    
    lines = []
    lines.append("📚 Docments Compliance Report")
    lines.append("=" * 50)
    lines.append(f"Total definitions: {len(results)}")
    lines.append(f"✅ Compliant: {len(compliant)}")
    lines.append(f"❌ Non-compliant: {len(non_compliant)}")
    if with_todos:
        lines.append(f"⚠️  With TODO placeholders: {len(with_todos)}")
    lines.append("")
    
    return lines

In [ ]:
#| export
def _generate_non_compliant_section(
    results: List[DocmentsCheckResult],  # Check results
    by_notebook: Dict[str, List[DocmentsCheckResult]]  # Results grouped by notebook
) -> List[str]:  # Lines of non-compliant section
    "Generate non-compliant definitions section of the report"
    non_compliant = [r for r in results if not r.is_compliant]
    lines = []
    
    if non_compliant:
        lines.append("❌ Non-compliant definitions:")
        lines.append("-" * 30)
        
        for nb in sorted(by_notebook.keys()):
            nb_results = by_notebook[nb]
            nb_non_compliant = [r for r in nb_results if not r.is_compliant]
            
            if nb_non_compliant:
                lines.append(f"\n📓 {nb}:")
                for r in nb_non_compliant:
                    lines.append(f"  ❌ {r.name}")
                    if not r.has_docstring:
                        lines.append("     - Missing docstring")
                    if r.missing_params:
                        lines.append(f"     - Missing docs for: {', '.join(r.missing_params)}")
    
    return lines

In [ ]:
#| export
def _generate_todos_section(
    results: List[DocmentsCheckResult],  # Check results
    by_notebook: Dict[str, List[DocmentsCheckResult]]  # Results grouped by notebook
) -> List[str]:  # Lines of TODOs section
    "Generate TODO placeholders section of the report"
    with_todos = [r for r in results if r.has_todos]
    lines = []
    
    if with_todos:
        lines.append("\n⚠️  Definitions with TODO placeholders:")
        lines.append("-" * 30)
        
        for nb in sorted(by_notebook.keys()):
            nb_results = by_notebook[nb]
            nb_todos = [r for r in nb_results if r.has_todos]
            
            if nb_todos:
                lines.append(f"\n📓 {nb}:")
                for r in nb_todos:
                    lines.append(f"  ⚠️  {r.name} ({r.todo_count} TODOs)")
    
    return lines

In [ ]:
#| export
def _generate_compliant_section(
    results: List[DocmentsCheckResult],  # Check results
    by_notebook: Dict[str, List[DocmentsCheckResult]]  # Results grouped by notebook
) -> List[str]:  # Lines of compliant section
    "Generate compliant definitions section of the report"
    compliant = [r for r in results if r.is_compliant]
    lines = []
    
    if compliant:
        lines.append("\n✅ Compliant definitions:")
        lines.append("-" * 30)
        for nb in sorted(by_notebook.keys()):
            nb_results = by_notebook[nb]
            nb_compliant = [r for r in nb_results if r.is_compliant and not r.has_todos]
            
            if nb_compliant:
                lines.append(f"\n📓 {nb}:")
                for r in nb_compliant:
                    lines.append(f"  ✅ {r.name}")
    
    return lines

In [ ]:
#| export
def generate_text_report(
    results: List[DocmentsCheckResult],  # Check results from check_project
    verbose: bool = False  # Include detailed information
) -> str:  # Formatted text report
    "Generate a human-readable text report of compliance results"
    # Group by notebook
    by_notebook = defaultdict(list)
    for r in results:
        by_notebook[r.notebook].append(r)
    
    lines = []
    
    # Add summary statistics
    lines.extend(_generate_summary_stats(results))
    
    # Add non-compliant section
    lines.extend(_generate_non_compliant_section(results, by_notebook))
    
    # Add TODOs section
    lines.extend(_generate_todos_section(results, by_notebook))
    
    # Add compliant section (if verbose)
    if verbose:
        lines.extend(_generate_compliant_section(results, by_notebook))
    
    return "\n".join(lines)

In [ ]:
#| export  
def generate_json_report(
    results: List[DocmentsCheckResult]  # Check results from check_project
) -> Dict[str, Any]:  # JSON-serializable report data
    "Generate a JSON report of compliance results"
    report = {
        "summary": {
            "total": len(results),
            "compliant": len([r for r in results if r.is_compliant]),
            "non_compliant": len([r for r in results if not r.is_compliant]),
            "with_todos": len([r for r in results if r.has_todos]),
            "total_todos": sum(r.todo_count for r in results)
        },
        "by_notebook": defaultdict(lambda: {"compliant": [], "non_compliant": [], "with_todos": []})
    }
    
    for r in results:
        entry = {
            "name": r.name,
            "type": r.type,
            "has_docstring": r.has_docstring,
            "missing_params": r.missing_params,
            "params_documented": r.params_documented,
            "return_documented": r.return_documented,
            "has_todos": r.has_todos,
            "todo_count": r.todo_count
        }
        
        if r.is_compliant:
            report["by_notebook"][r.notebook]["compliant"].append(entry)
        else:
            report["by_notebook"][r.notebook]["non_compliant"].append(entry)
            
        if r.has_todos:
            report["by_notebook"][r.notebook]["with_todos"].append(entry)
    
    # Convert defaultdict to regular dict for JSON serialization
    report["by_notebook"] = dict(report["by_notebook"])
    
    return report

In [ ]:
# Test the report generator
results = check_project()
print(generate_text_report(results, verbose=True))

📚 Docments Compliance Report
Total definitions: 24
✅ Compliant: 24
❌ Non-compliant: 0


✅ Compliant definitions:
------------------------------

📓 00_core.ipynb:
  ✅ DocmentsCheckResult
  ✅ extract_param_docs
  ✅ check_return_doc
  ✅ count_todos_in_docs
  ✅ check_has_docstring
  ✅ check_params_documentation
  ✅ determine_compliance
  ✅ check_definition
  ✅ check_notebook
  ✅ check_function

📓 01_scanner.ipynb:
  ✅ get_export_cells
  ✅ extract_definitions
  ✅ scan_notebook
  ✅ scan_project

📓 02_report.ipynb:
  ✅ check_project
  ✅ _generate_summary_stats
  ✅ _generate_non_compliant_section
  ✅ _generate_todos_section
  ✅ _generate_compliant_section
  ✅ generate_text_report
  ✅ generate_json_report

📓 03_autofix.ipynb:
  ✅ generate_fixed_source
  ✅ fix_notebook

📓 04_cli.ipynb:
  ✅ main


In [ ]:
# Test JSON report
json_report = generate_json_report(results)
print(json.dumps(json_report, indent=2))

{
  "summary": {
    "total": 24,
    "compliant": 24,
    "non_compliant": 0,
    "with_todos": 0,
    "total_todos": 0
  },
  "by_notebook": {
    "00_core.ipynb": {
      "compliant": [
        {
          "name": "DocmentsCheckResult",
          "type": "ClassDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {},
          "return_documented": true,
          "has_todos": false,
          "todo_count": 0
        },
        {
          "name": "extract_param_docs",
          "type": "FunctionDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {
            "source": true
          },
          "return_documented": true,
          "has_todos": false,
          "todo_count": 0
        },
        {
          "name": "check_return_doc",
          "type": "FunctionDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {
            "source": 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
# Test TODO tracking with a function that has placeholder docs
from cjm_nbdev_docments.core import check_definition

test_with_todos = '''def example_func(
    x: int,  # TODO: Add description
    y: str  # TODO: Add description  
) -> bool:  # TODO: Add return description
    "TODO: Add function description"
    return True'''

test_def = {
    'name': 'example_func',
    'type': 'FunctionDef', 
    'source': test_with_todos,
    'notebook': 'test.ipynb',
    'args': [
        {'name': 'x', 'annotation': 'int'},
        {'name': 'y', 'annotation': 'str'}
    ],
    'returns': 'bool'
}

result = check_definition(test_def)
print(f"Function: {result.name}")
print(f"Compliant: {result.is_compliant}")
print(f"Has TODOs: {result.has_todos}")
print(f"TODO count: {result.todo_count}")

# Test current project report
print("\n" + "="*50)
print("Current project with TODO tracking:")
results = check_project()
print(generate_text_report(results))

Function: example_func
Compliant: True
Has TODOs: True
TODO count: 4

Current project with TODO tracking:
📚 Docments Compliance Report
Total definitions: 24
✅ Compliant: 24
❌ Non-compliant: 0

